In [ ]:
import numpy as np
import math

class ID3():
    def __init__(self):
        
    
    @staticmethod
    def entropy(data):
        '''
        输入数据data,输出其经验熵'''
        n=len(data)   #数据个数
        label_dict={}
        for i in range(n):
            label_dict[data[i][-1]]=label_dict.get(data[i][-1],0)+1
        k=len(label_dict)  #类别个数
        ent=0
        for n_k in label_dict.values():
            ent+= n_k/n * math.log(n_k/n,2)
        return -ent
    
    @staticmethod
    def conditional_entropy(data,a):
        '''
        输入数据data和用来分类的特征a(即数据的第a列)'''
        n=len(data)   #数据个数
        con_ent=0
        new_data=ID3.data_divide(data,a)
        for curr_data in new_data:
            con_ent+= len(curr_data)/n * ID3.entropy(curr_data)        
        return -con_ent
    
    @staticmethod
    def data_divide(data,a):
        '''
        根据第a列特征将数据划分'''
        new_data={}
        for curr_data in data:
            new_data[curr_data[a]]=new_data.get(curr_data[a],[]).append(curr_data)
        return list(new_data.values())
    
    

In [ ]:
datasets = [['青年', '否', '否', '一般', '否'],
           ['青年', '否', '否', '好', '否'],
           ['青年', '是', '否', '好', '是'],
           ['青年', '是', '是', '一般', '是'],
           ['青年', '否', '否', '一般', '否'],
           ['中年', '否', '否', '一般', '否'],
           ['中年', '否', '否', '好', '否'],
           ['中年', '是', '是', '好', '是'],
           ['中年', '否', '是', '非常好', '是'],
           ['中年', '否', '是', '非常好', '是'],
           ['老年', '否', '是', '非常好', '是'],
           ['老年', '否', '是', '好', '是'],
           ['老年', '是', '否', '好', '是'],
           ['老年', '是', '否', '非常好', '是'],
           ['老年', '否', '否', '一般', '否'],
           ]
labels = [u'年龄', u'有工作', u'有自己的房子', u'信贷情况', u'类别']